# Telemetry History Analysis

This notebook demonstrates how to consume the `history_summary.*` artefacts
produced by continuous integration and extend the analysis locally.

## Load History Snapshot

The `telemetry-report` artefact includes `history_summary.csv` and
`history_summary.html`. We'll load the CSV and pivot the data for quick analysis.

In [ ]:
from pathlib import Path
import pandas as pd

HISTORY_CSV = Path('docs/examples/analytics/data/tuner_reports/history_summary.csv')
if HISTORY_CSV.exists():
    history = pd.read_csv(HISTORY_CSV)
else:
    # Fallback synthetic history if CI artefact not present locally
    history = pd.DataFrame([
        {'snapshot': '2024-11-10T000000Z', 'algorithm': 'random', 'scenario': 'SampleScenario', 'best_objective': 6.2, 'mean_objective': 6.0, 'runs': 2},
        {'snapshot': '2024-11-11T000000Z', 'algorithm': 'random', 'scenario': 'SampleScenario', 'best_objective': 6.8, 'mean_objective': 6.4, 'runs': 2},
        {'snapshot': '2024-11-12T000000Z', 'algorithm': 'grid', 'scenario': 'SampleScenario', 'best_objective': 7.1, 'mean_objective': 6.8, 'runs': 2},
        {'snapshot': '2024-11-13T000000Z', 'algorithm': 'grid', 'scenario': 'SampleScenario', 'best_objective': 7.5, 'mean_objective': 7.1, 'runs': 2},
    ])
history

## Compute Deltas
We can compute day-over-day improvements for each algorithm/scenario pair.

In [ ]:
history = history.sort_values(['scenario', 'algorithm', 'snapshot']).reset_index(drop=True)
history['best_delta'] = history.groupby(['scenario', 'algorithm'])['best_objective'].diff()
history

## Visualize Trends
Use Altair to plot the best objective trend per scenario.

In [ ]:
try:
    import altair as alt
except ImportError:
    chart = 'Altair not available; install altair to see the chart.'
else:
    chart = (
        alt.Chart(history)
        .mark_line(point=True)
        .encode(
            x='snapshot:N',
            y='best_objective:Q',
            color='algorithm:N',
            column='scenario:N'
        )
    )
chart

## Next Steps
- Merge med42 or regression scenario reports into the history before comparing.
- Use run-level telemetry (from `runs.sqlite`) to inspect KPI deltas for each snapshot.